In [1]:
from keras.applications.xception import Xception
from keras.applications import imagenet_utils
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras import optimizers
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

import numpy as np
import argparse

Using TensorFlow backend.


In [2]:
modelX = Xception(weights='imagenet', include_top=True)

In [3]:
# if we are using the InceptionV3 or Xception networks, then we
# need to set the input shape to (299x299) [rather than (224x224)]
# and use a different image processing function
inputShape = (299, 299)
preprocess = preprocess_input

In [4]:
# load the input image using the Keras helper utility while ensuring
# the image is resized to `inputShape`, the required input dimensions
# for the ImageNet pre-trained network

image = load_img('data/train/dogs/dog.0.jpg', target_size=inputShape)
image = img_to_array(image)
 
# our input image is now represented as a NumPy array of shape
# (inputShape[0], inputShape[1], 3) however we need to expand the
# dimension by making the shape (1, inputShape[0], inputShape[1], 3)
# so we can pass it through thenetwork
image = np.expand_dims(image, axis=0)
 
# pre-process the image using the appropriate function based on the
# model that has been loaded (i.e., mean subtraction, scaling, etc.)
image = preprocess(image)

In [5]:
preds = modelX.predict(image)
P = imagenet_utils.decode_predictions(preds)
 
# loop over the predictions and display the rank-5 predictions +
# probabilities to our terminal
for (i, (imagenetID, label, prob)) in enumerate(P[0]):
    print("{}. {}: {:.2f}%".format(i + 1, label, prob * 100))

1. affenpinscher: 20.79%
2. Yorkshire_terrier: 16.80%
3. silky_terrier: 12.21%
4. cairn: 11.75%
5. Tibetan_terrier: 3.25%


In [6]:
print(len(modelX.layers)) 

134


In [14]:
base_model = applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape = (224,224,3))

In [15]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [16]:
for layer in base_model.layers:
    layer.trainable = False

In [17]:
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(2, activation="softmax") (x)

model_final = Model(input=base_model.input, output=predictions)
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [18]:
model_final.compile(loss="categorical_crossentropy", optimizer=optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255)
val_datagen = ImageDataGenerator(rescale=1. / 255)

train_data_dir = 'data/train'
validation_data_dir = "data/validation"
nb_validation_samples = 1000
nb_train_samples = 1000
epochs = 5
batch_size = 20
img_width, img_height = 224, 224

train_generator = train_datagen.flow_from_directory(
                train_data_dir,
                target_size=(img_width, img_height),
                batch_size=batch_size,
                class_mode='categorical',
                shuffle=False)

val_generator = val_datagen.flow_from_directory(
                                validation_data_dir,
                                target_size = (img_height, img_width),
                                batch_size=batch_size,
                                class_mode = "categorical")

checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


# Train the model 
model_final.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples//batch_size,
        epochs = epochs,
        validation_data=val_generator,
        validation_steps=nb_validation_samples//batch_size,
        #nb_val_samples=nb_validation_samples//batch_size,
        verbose=1,
        workers=4,
        callbacks = [checkpoint, early])

Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/5
50/50 [==============================] - 77s - loss: 0.6220 - acc: 0.6250 - val_loss: 0.9822 - val_acc: 0.5030
Epoch 2/5
50/50 [==============================] - 77s - loss: 0.8003 - acc: 0.5650 - val_loss: 0.8240 - val_acc: 0.5170
Epoch 3/5
50/50 [==============================] - 75s - loss: 0.8253 - acc: 0.4170 - val_loss: 0.6892 - val_acc: 0.5150
Epoch 4/5
50/50 [==============================] - 75s - loss: 0.6732 - acc: 0.5360 - val_loss: 0.7237 - val_acc: 0.5060
Epoch 5/5
 2/50 [>.............................] - ETA: 36s - loss: 1.0294 - acc: 0.0000e+00

In [ ]:
print('abc')